In [1]:
from mesmerize_core import *
import numpy as np
from copy import deepcopy
import pandas as pd

2023-01-08 19:52:56.121755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:52:56.205878: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 19:52:56.584962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2023-01-08 19:52:56.585014: W tensorflow/stream_executor/platform/default/dso_

**You will need `fastplotlib` installed for the visualizations**

In [2]:
from fastplotlib import ImageWidget
from ipywidgets import VBox, IntSlider, Layout

In [3]:
pd.options.display.max_colwidth = 120

# Paths
These are the only variables you will need to modify in this demo notebook. You will need to set the paths according to your own `caiman_data` dir path

Explanation:

`set_parent_raw_data_path()` - This function from `mesmerize_core` sets the **top level raw data directory**. It is generally the top level directory for your raw experimental data. This allows you to move your experiment directory structure between computers, as long as you keep everything under the parent path the same.

For example,

On Linux based systems if you have your experimental data in the following dir:

`/data/my_name/exp_top_level/....`

You could set `/data/my_name` as the "parent raw data path", and you can then move `exp_top_level/...` between computers.

On windows:

`D:/my_name/exp_top_level/...`

You could set `D:/my_name` as the parent raw data path, and you can then move `exp_top_level/...` between computers.

In [4]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushalk/caiman_data/")

PosixPath('/home/kushalk/caiman_data')

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

**We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems.**

In [5]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [6]:
# create a new batch
df = create_batch(batch_path)
# to load existing batches use `load_batch()`
# df = load_batch(batch_path)

# View the dataframe

It is empty with the appropriate columns for mesmerize

In [7]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid


# Path to the input movie

An input movie must be anywhere within `raw data path` or `batch path`. We will use the Sue 2p example.

In [8]:
movie_path = get_parent_raw_data_path().joinpath("example_movies/Sue_2x_3000_40_-46.tif")

# Motion correction parameters

Parameters for all algos have the following structure:

```python
{"main": {... params directly passed to caiman}}
```

In [9]:
# We will start with one version of parameters
mcorr_params1 =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [24, 24],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# Add a "batch item", this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

In [10]:
# add an item to the batch
df.caiman.add_item(
    algo='mcorr',
    input_movie_path=movie_path,
    params=mcorr_params1,
    item_name=movie_path.stem,  # filename of the movie, but can be anything
)

df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,None,7e9eaf1a-b101-457d-8fa9-44a31516cdbf


## We can now see that there is one item, a.k.a. row or pandas `Series`, in the batch dataframe, we can add another item with the same input movie but with different parameters.

### **When adding batch items with the same `input_movie_path` (i.e. same input movie but different parameters) it is useful to give them the same `item_name`.**

In [11]:
# We create another set of params, useful for gridsearches for example
mcorr_params2 =\
{
  'main':
    {
        'max_shifts': [24, 24],
        'strides': [24, 24],
        'overlaps': [12, 12],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# add other param variant to the batch
df.caiman.add_item(
  algo='mcorr',
  item_name=movie_path.stem,
  input_movie_path=movie_path,
  params=mcorr_params2
)

df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,None,7e9eaf1a-b101-457d-8fa9-44a31516cdbf
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,None,a1f3f2af-acda-465e-a05c-5efeb9b98236


## We can see that there are two batch items for the same input movie.

### We can also use a `for` loop to add multiple different parameter variants more efficiently.

In [12]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(mcorr_params2)

# some variants of max_shifts
for shifts in [1, 6, 12]: 
    # deep copy is the safest way to copy dicts
    new_params = deepcopy(new_params)
    
    # assign the "max_shifts"
    new_params["main"]["max_shifts"] = (shifts, shifts)
    
    df.caiman.add_item(
      algo='mcorr',
      item_name=movie_path.stem,
      input_movie_path=movie_path,
      params=new_params
    )

In [13]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,None,7e9eaf1a-b101-457d-8fa9-44a31516cdbf
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,None,a1f3f2af-acda-465e-a05c-5efeb9b98236
2,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",None,None,e413e478-378d-4e69-8c60-960b5a6004cd
3,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",None,None,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb
4,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,None,7dd81f6a-79a6-465d-bed0-590ec7521718


## Now we can see that there are many parameter variants, but it is not easy to see the differences in parameters between the rows that have the same `item_name`.

### We can use the `caiman.get_params_diffs()` extension to see the unique parameters between rows with the same `item_name`

In [14]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

/tmp/ipykernel_662129/1437176290.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])


0    {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24)}
1    {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12)}
2      {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12)}
3      {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12)}
4    {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12)}
Name: params, dtype: object

# Indexing rows and running batch item(s)
#### You can run a single batch item by calling `caiman.run()` on a `Series` (row) of the DataFrame. One way to get the row is integer indexing using `df.iloc[index]`

In [15]:
# get the first batch item
row = df.iloc[0]

### You can see how the various `pandas.Series` extensions are accessible at the level of dataframe rows.

Move the cursor to the end of the following line and press `Tab` on your keyboard. You can select the `caiman.run()` function and press `Shift + Tab` to see the docstring. You can also instead refer to the API docs. https://mesmerize-core.readthedocs.io/en/latest/api/common.html#mesmerize_core.CaimanSeriesExtensions

Note tab completion doesn't work if you use `df.iloc[i].caiman.<method_name>`, you need to apply the indexer to see the docstring.

In [16]:
row.caiman.

SyntaxError: invalid syntax (1726614793.py, line 1)

# Run a single batch item

Run the row that we have selected above, on Linux & Mac it will run in subprocess but on Windows it will run in the local kernel. If using the subprocess backend you can use `run(wait=False)` to not block the current kernel when running.

In [17]:
# run the first "batch item"
# this will run in a subprocess by default on Linux & Mac
# on windows it will run locally
process = row.caiman.run()

# reload dataframe from disk when done
df = df.caiman.reload_from_disk()

2023-01-08 19:53:18.124257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:53:18.206996: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 19:53:18.574628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

starting mc


mc finished successfully!
computing projections
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf1a-b101-457d-8fa9-44a31516cdbf-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf1a-b101-457d-8fa9-44a31516cdbf-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf1a-b101-457d-8fa9-44a31516cdbf-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmapDecode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf1a-b101-457d-8fa9-44a31516cdbf-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmapDecode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf

100%|██████████| 1/1 [00:00<00:00, 12.05it/s]

100%|██████████| 1/1 [00:00<00:00,  8.00it/s]


finished computing correlation image


# Use a loop to run multiple batch items.

`df.iterrows()` iterates through rows and returns the numerical index and row for each iteration

In [18]:
for i, row in df.iterrows():
    if not i > 0: # skip the first item since we've run it already
        continue
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

2023-01-08 19:53:28.780253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:53:28.864452: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 19:53:29.234716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

starting mc


mc finished successfully!
computing projections
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/a1f3f2af-acda-465e-a05c-5efeb9b98236/a1f3f2af-acda-465e-a05c-5efeb9b98236-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/a1f3f2af-acda-465e-a05c-5efeb9b98236/a1f3f2af-acda-465e-a05c-5efeb9b98236-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/a1f3f2af-acda-465e-a05c-5efeb9b98236/a1f3f2af-acda-465e-a05c-5efeb9b98236-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/a1f3f2af-acda-465e-a05c-5efeb9b98236/a1f3f2af-acda-465e-a05c-5efeb9b98236-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmapDecode mmap filename /home/kushalk/caiman_data/mesmerize-batch/a1f3f2af-acda-465e-a05c-5efeb9b98236/a1f3f

100%|██████████| 1/1 [00:00<00:00,  8.31it/s]


finished computing correlation image


...

2023-01-08 19:54:03.608595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:54:03.692729: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 19:54:04.063311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

starting mc


mc finished successfully!
computing projections
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7dd81f6a-79a6-465d-bed0-590ec7521718/7dd81f6a-79a6-465d-bed0-590ec7521718-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7dd81f6a-79a6-465d-bed0-590ec7521718/7dd81f6a-79a6-465d-bed0-590ec7521718-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7dd81f6a-79a6-465d-bed0-590ec7521718/7dd81f6a-79a6-465d-bed0-590ec7521718-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7dd81f6a-79a6-465d-bed0-590ec7521718/7dd81f6a-79a6-465d-bed0-590ec7521718-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmapDecode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7dd81f6a-79a6-465d-bed0-590ec7521718/7dd81

100%|██████████| 1/1 [00:00<00:00,  8.51it/s]


finished computing correlation image


# Reload the DataFrame to see the outputs information for the mcorr batch item
### It is necessary to ALWAYS use `df = df.caiman.reload_from_disk()` after running a single batch item or a loop of batch items. You must not add new batch items until you reload it if you have ran items!

In [19]:
df = df.caiman.reload_from_disk()

## We can see that the `outputs` column has been filled in

In [20]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf1a-b101-457d-8fa9-44a31516cdbf_mean_projection.n...,None,7e9eaf1a-b101-457d-8fa9-44a31516cdbf
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': a1f3f2af-acda-465e-a05c-5efeb9b98236/a1f3f2af-acda-465e-a05c-5efeb9b98236_mean_projection.n...,None,a1f3f2af-acda-465e-a05c-5efeb9b98236
2,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': e413e478-378d-4e69-8c60-960b5a6004cd/e413e478-378d-4e69-8c60-960b5a6004cd_mean_projection.n...,None,e413e478-378d-4e69-8c60-960b5a6004cd
3,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb_mean_projection.n...,None,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb
4,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 7dd81f6a-79a6-465d-bed0-590ec7521718/7dd81f6a-79a6-465d-bed0-590ec7521718_mean_projection.n...,None,7dd81f6a-79a6-465d-bed0-590ec7521718


# Check if the algorithm ran successfully for a item

In [21]:
# True if the algo ran succesfully
df.iloc[0]["outputs"]["success"]

True

# Visualization using `fastplotlib`
You will need `fastplotlib` installed for this, see https://github.com/kushalkolar/fastplotlib

# Get the input movie and mcorr so we can visualize them

Note that you DO NOT need to manually work with file paths. For tiff input files it returns it as a memmaped array (if possible) with lazy loading. It will try to use a mesmerize `LazyArray` if the file cannot be memmaped.

In [22]:
# you can change the index to look at the mcorr results of different batch items
index = 0

# get input movie as memmap
input_movie = df.iloc[index].caiman.get_input_movie()

# load mcorr output movie, also as a memmaped array
mcorr_movie = df.iloc[index].mcorr.get_output()

Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf1a-b101-457d-8fa9-44a31516cdbf-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap


# Visualize raw & MCorr movie side-by-side

### fastplotlib `ImageWidget` to visualize raw & mcorr movie side by side

`ImageWidget` assumes `"txy"` dimension order by default for 2D movies. You can set other orders using the `dims_order` kwarg

In [23]:
mcorr_iw = ImageWidget(
    data=[input_movie, mcorr_movie], 
    vmin_vmax_sliders=True, 
    cmap="gnuplot2"
)
mcorr_iw.show()

RFBOutputContext()

# Frame averaging over a window

This makes it easier to visually inspect motion

In [24]:
# window function on the "t" (time) dimension, using mean of 17 frames
mcorr_iw.window_funcs = {"t": (np.mean, 17)}

## Close the canvas to free up GPU processing time, not necessary if you have a powerful GPU

In [24]:
mcorr_iw.plot.canvas.close()

## With `ImageWidget` you can view all 5 mcorr results simultaneously!

### This depends on your hard drive's capabilities

In [25]:
# first item is just the raw movie
movies = [df.iloc[0].caiman.get_input_movie()]

# subplot titles
subplot_names = ["raw"]

# we will use the mean images later
means = [df.iloc[0].caiman.get_projection("mean")]

# add all the mcorr outputs to the list
for i, row in df.iterrows():
    # add to the list of movies to plot
    movies.append(row.mcorr.get_output())
    
    # the unique params for this item
    params = diffs.iloc[i]
    
    # subplot title to show dataframe index
    subplot_names.append(f"ix: {i}")
    
    # mean images which we'll use later
    means.append(row.caiman.get_projection("mean"))

# create the widget
mcorr_iw_multiple = ImageWidget(
    data=movies,  # list of movies
    window_funcs={"t": (np.mean, 17)}, # window_funcs is also a kwarg
    vmin_vmax_sliders=True,
    names=subplot_names,  # subplot names used for titles
    cmap="gnuplot2"
)
mcorr_iw_multiple.show()

Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7e9eaf1a-b101-457d-8fa9-44a31516cdbf/7e9eaf1a-b101-457d-8fa9-44a31516cdbf-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/a1f3f2af-acda-465e-a05c-5efeb9b98236/a1f3f2af-acda-465e-a05c-5efeb9b98236-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/e413e478-378d-4e69-8c60-960b5a6004cd/e413e478-378d-4e69-8c60-960b5a6004cd-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/7dd81f6a-79a6-465d-bed0-590ec7521718/7dd81f6a-79a6-465d-bed0-590ec7521718-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_

RFBOutputContext()

In [26]:
df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])

/tmp/ipykernel_662129/3787477196.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])


0    {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24)}
1    {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12)}
2      {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12)}
3      {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12)}
4    {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12)}
Name: params, dtype: object

### Modify the `window_funcs` at any time

In [30]:
mcorr_iw_multiple.window_funcs["t"].window_size = 23

## There is some motion on the left side of of `ix: 2` at timepoint `1452`, `2037` and a few others. This will be more obvious if we substract a mean image from each frame. You can use `frame_apply` to apply a function before displaying frames in the `ImageWidget`

This can be combined with `window_funcs` or used by itself. If used in combination with `window_funcs`, the window functions are computed first and then fed to `frame_apply`.

For this example the `frame_apply` functions subtract the mean image for each movie.

General form:

```python
{
    data_ix: function() # returns 2D frame
    ...
}
```

In [31]:
subtract_means = {
    0: lambda x: x - means[0],
    1: lambda x: x - means[1],
    2: lambda x: x - means[2],
    3: lambda x: x - means[3],
    4: lambda x: x - means[4],
    5: lambda x: x - means[5]
}

In [32]:
mcorr_iw_multiple.frame_apply = subtract_means

### Different colormaps can make the motion more obvious

In [34]:
for sp in mcorr_iw_multiple.plot:
    sp.graphics[0].cmap = "jet"

In [35]:
# disable frame apply
mcorr_iw_multiple.frame_apply = dict()

# ix `3` seems to work the best so we will cleanup the DataFrame and remove all other items.

### You can remove batch items (i.e. rows) using `df.caiman.remove_item(<item_uuid>)`

**Note that this also cleans up the output data in the batch directory!**

In [36]:
# make a list of rows we want to keep using the uuids
rows_keep = [df.iloc[3].uuid]
rows_keep

['3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb']

In [37]:
for i, row in df.iterrows():
    if row.uuid not in rows_keep:
        df.caiman.remove_item(row.uuid)

df

/home/kushalk/repos/mesmerize-core/mesmerize_core/caiman_extensions/common.py:206: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
/home/kushalk/repos/mesmerize-core/mesmerize_core/caiman_extensions/common.py:206: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
/home/kushalk/repos/mesmerize-core/mesmerize_core/caiman_extensions/common.py:206: FutureWarning: You are trying to use the following experi

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb_mean_projection.n...,None,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb


### As you can see above, the numerical index changed for what was previously item 3. Indices are always reset when you use `caiman.remove_item()`. However, UUIDs are always maintained.

# CNMF

## Continue from mcorr above and perform CNMF using the good mcorr output.

First, the params for CNMF. Put the CNMF params within the `main` key, `refit` is if you want to run CNMF for a second iteration.

In [38]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.85,
            'rf': 15,
            'stride': 6, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'ssub': 1,
            'tsub': 1,
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

### Add a single cnmf item to the batch

In [39]:
# add a batch item
df.caiman.add_item(
    algo='cnmf', # algo is cnmf
    input_movie_path=df.iloc[0],  # use mcorr output from a completed batch item
    params=params_cnmf,
    item_name=df.iloc[0]["item_name"], # use the same item name
)

### Just like with motion correction, we can use loops to add multiple parameter variants. This is useful to perform a parameter search to find the params that work best for your dataset. Here I will use `itertools.product` which is better than deeply nested loops.

In [40]:
from itertools import product

# variants of several parameters
gSig_variants = [6, 8]
K_variants = [4, 8]
merge_thr_variants = [0.8, 0.95]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(gSig_variants, K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for gSig, K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name=df.iloc[0]["item_name"],
        input_movie_path=df.iloc[0],
        params=new_params_cnmf
    )

### See that there are a lot of new cnmf batch items

In [41]:
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb_mean_projection.n...,None,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb
1,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",None,None,581150df-4149-49cc-9346-89fdec8cefba
2,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,None,ef67d1e8-18ee-4739-804c-4d57d48c7caf
3,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",None,None,96d56a2e-f085-42eb-a426-5319b5a68cc4
4,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,None,8a55be31-b305-405e-9f9c-d1944b1e077c
5,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",None,None,419aba70-8e19-4fb9-ac0d-e875eeb11bf0
6,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,None,57055675-4e8b-498b-826c-5970be83a016
7,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",None,None,a5fa3dae-8f26-4cec-904d-7f20eccb90e9
8,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,None,77e341f0-c41b-4f22-a65c-17c15a2c0dec
9,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 't...",None,None,ab198be6-dd2a-41ce-889e-4ff41c6ebc12


## Since it is difficult to see the different parameter variants above, we can just view the diffs

### The index numbers on the diffs correspond to the indices in the parent DataFrame above

In [42]:
df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])

/tmp/ipykernel_662129/3330105589.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])


1    {'K': 4, 'gSig': (4, 4), 'merge_thr': 0.85}
2     {'K': 4, 'gSig': (6, 6), 'merge_thr': 0.8}
3    {'K': 4, 'gSig': (6, 6), 'merge_thr': 0.95}
4     {'K': 8, 'gSig': (6, 6), 'merge_thr': 0.8}
5    {'K': 8, 'gSig': (6, 6), 'merge_thr': 0.95}
6     {'K': 4, 'gSig': (8, 8), 'merge_thr': 0.8}
7    {'K': 4, 'gSig': (8, 8), 'merge_thr': 0.95}
8     {'K': 8, 'gSig': (8, 8), 'merge_thr': 0.8}
9    {'K': 8, 'gSig': (8, 8), 'merge_thr': 0.95}
Name: params, dtype: object

### Run the added `cnmf` batch items, they start at index 1 as we can see from the DataFrame.

In [43]:
for i, row in df.iterrows():
    if i < 1:
        continue
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = load_batch(df.paths.get_batch_path())

2023-01-08 19:58:51.776921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:58:51.862115: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 19:58:52.238262: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                          Sue_2x_3000_40_-46
input_movie_path    3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
comments                                                         None
uuid                             581150df-4149-49cc-9346-89fdec8cefba
Name: 1, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, 'rval_thr': 0.7, 'use_cnn': True, 'min_cnn_thr': 0.8, 'cnn_lowest': 0.1, 'decay_time': 0.4}, 'refit': True}
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b8

2023-01-08 19:59:04.273692: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-08 19:59:04.273729: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: hantman-calcium
2023-01-08 19:59:04.273735: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: hantman-calcium
2023-01-08 19:59:04.273975: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-01-08 19:59:04.274001: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2023-01-08 19:59:04.274006: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.161.3
2023-01-08 19:59:04.274285: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU

5/5 [==============================] - 0s 11ms/step


2023-01-08 19:59:07.880903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:59:07.963181: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 19:59:08.333389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

...

2023-01-08 20:01:24.177149: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 20:01:24.261859: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 20:01:24.639791: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib6

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                          Sue_2x_3000_40_-46
input_movie_path    3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
comments                                                         None
uuid                             ab198be6-dd2a-41ce-889e-4ff41c6ebc12
Name: 9, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, 'rval_thr': 0.7, 'use_cnn': True, 'min_cnn_thr': 0.8, 'cnn_lowest': 0.1, 'decay_time': 0.4}, 'refit': True}
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b8

making memmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/ab198be6-dd2a-41ce-889e-4ff41c6ebc12_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
performing CNMF
fitting images
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/ab198be6-dd2a-41ce-889e-4ff41c6ebc12_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/ab198be6-dd2a-41ce-889e-4ff41c6ebc12_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/ab198be6-dd2a-41ce-889e-4ff41c6ebc12_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushalk/caiman_data/mesmerize-batch/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/ab198be6-dd2a-41ce-889e-4ff41c6ebc12_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_

2023-01-08 20:01:42.689517: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-08 20:01:42.689541: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: hantman-calcium
2023-01-08 20:01:42.689544: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: hantman-calcium
2023-01-08 20:01:42.689647: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-01-08 20:01:42.689661: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2023-01-08 20:01:42.689663: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.161.3
2023-01-08 20:01:42.689832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU

GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/kushalk/caiman_data/model/cnn_model.json
12/12 [==============================] - 0s 10ms/step


### We now have CNMF outputs

In [44]:
df = df.caiman.reload_from_disk()
df[df["algo"] == "cnmf"]

,algo,item_name,input_movie_path,params,outputs,comments,uuid
1,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",{'mean-projection-path': 581150df-4149-49cc-9346-89fdec8cefba/581150df-4149-49cc-9346-89fdec8cefba_mean_projection.n...,None,581150df-4149-49cc-9346-89fdec8cefba
2,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': ef67d1e8-18ee-4739-804c-4d57d48c7caf/ef67d1e8-18ee-4739-804c-4d57d48c7caf_mean_projection.n...,None,ef67d1e8-18ee-4739-804c-4d57d48c7caf
3,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': 96d56a2e-f085-42eb-a426-5319b5a68cc4/96d56a2e-f085-42eb-a426-5319b5a68cc4_mean_projection.n...,None,96d56a2e-f085-42eb-a426-5319b5a68cc4
4,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': 8a55be31-b305-405e-9f9c-d1944b1e077c/8a55be31-b305-405e-9f9c-d1944b1e077c_mean_projection.n...,None,8a55be31-b305-405e-9f9c-d1944b1e077c
5,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': 419aba70-8e19-4fb9-ac0d-e875eeb11bf0/419aba70-8e19-4fb9-ac0d-e875eeb11bf0_mean_projection.n...,None,419aba70-8e19-4fb9-ac0d-e875eeb11bf0
6,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': 57055675-4e8b-498b-826c-5970be83a016/57055675-4e8b-498b-826c-5970be83a016_mean_projection.n...,None,57055675-4e8b-498b-826c-5970be83a016
7,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",{'mean-projection-path': a5fa3dae-8f26-4cec-904d-7f20eccb90e9/a5fa3dae-8f26-4cec-904d-7f20eccb90e9_mean_projection.n...,None,a5fa3dae-8f26-4cec-904d-7f20eccb90e9
8,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': 77e341f0-c41b-4f22-a65c-17c15a2c0dec/77e341f0-c41b-4f22-a65c-17c15a2c0dec_mean_projection.n...,None,77e341f0-c41b-4f22-a65c-17c15a2c0dec
9,cnmf,Sue_2x_3000_40_-46,3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb/3b88d12f-c1f6-4bf3-b004-63c6eaeefbfb-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 't...",{'mean-projection-path': ab198be6-dd2a-41ce-889e-4ff41c6ebc12/ab198be6-dd2a-41ce-889e-4ff41c6ebc12_mean_projection.n...,None,ab198be6-dd2a-41ce-889e-4ff41c6ebc12


In [45]:
# see which batch items completed succcessfully
df[df["algo"] == "cnmf"]["outputs"].apply(lambda x: x["success"])

1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: outputs, dtype: bool